# Logistic Regression vs LogRegCCD on Synthetic Data

In [ ]:
import os
if not os.path.exists("./notebooks"):
    %cd ..
    %cd ..

import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import pandas as pd
from dataclasses import dataclass
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from src.log_reg_ccd import LogRegCCD
from src.data.data_loader import SyntheticDataLoader
from src.data.dataset_interface import DataInterface
import src.measures as measure

## User specified variables

In [ ]:
random_seed = 42
chosen_experiment = 'Default' # Chosen experiment title - check under Experiments params section
run_analyses = True # wheter to run parameters analyses
results_dir = 'results' # If not None - directory to save te analyses plots

## Utils

In [ ]:
@dataclass
class ExperimentParams:
    title: str
    n: int
    p: float
    d: int
    g: float
    eps: float = 1e-3
    lam_max: float = 10.0
    lam_count: int = 100
    k_fold: int = 10

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
DEFAULT_N = 1000
DEFAULT_P = 0.5
DEFAULT_D = 50
DEFAULT_G = 0

In [ ]:
aucroc = measure.AUCROC()
ba = measure.BalancedAccuracy()

## Experiments params

In [ ]:
experiments = [
    
    ExperimentParams(title='Default', n=DEFAULT_N, p=DEFAULT_P, d=DEFAULT_D, g=DEFAULT_G),

    ExperimentParams(title='Few samples', n=100, p=DEFAULT_P, d=DEFAULT_D, g=DEFAULT_G),
    ExperimentParams(title='Lots of samples', n=10000, p=DEFAULT_P, d=DEFAULT_D, g=DEFAULT_G),

    ExperimentParams(title='Slightly imbalanced', n=DEFAULT_N, p=0.3, d=DEFAULT_D, g=DEFAULT_G),
    ExperimentParams(title='Highly impalanced', n=DEFAULT_N, p=0.1, d=DEFAULT_D, g=DEFAULT_G),

    ExperimentParams(title='Few features', n=DEFAULT_N, p=DEFAULT_P, d=10, g=DEFAULT_G),
    ExperimentParams(title='Lots of features', n=DEFAULT_N, p=DEFAULT_P, d=500, g=DEFAULT_G),
    ExperimentParams(title='More features than samples', n=DEFAULT_N, p=DEFAULT_P, d=1500, g=DEFAULT_G),

    ExperimentParams(title='Weak correlation', n=DEFAULT_N, p=DEFAULT_P, d=DEFAULT_D, g=0.1),
    ExperimentParams(title='Moderate correlation', n=DEFAULT_N, p=DEFAULT_P, d=DEFAULT_D, g=0.5),
    ExperimentParams(title='Strong correlation', n=DEFAULT_N, p=DEFAULT_P, d=DEFAULT_D, g=0.8),

    ExperimentParams(title='Moderate correlation, lots of features', n=DEFAULT_N, p=DEFAULT_P, d=500, g=0.5),
    ExperimentParams(title='Strong correlation, lots of features', n=DEFAULT_N, p=DEFAULT_P, d=500, g=0.8),
]

experiment = next(exp for exp in experiments if exp.title == chosen_experiment)

## Helpers functions

In [ ]:
def test_model(model, X, y, save_path=None, ccd=True, pr=True):
    """ Evaluate model according to chosen metrics"""

    results = {}

    y_pred = model.predict(X)
    if ccd:
        y_pred_proba = model.predict_proba(X)
        results['coefficients'] = model.best_beta
    else:
        y_pred_proba = model.predict_proba(X)[:,1]
        results['coefficients'] = np.concatenate((model.intercept_, model.coef_[0]))

    results['accuracy'] = accuracy_score(y, y_pred)
    results['auc_roc'] = aucroc(y, y_pred_proba)
    results['balanced_accuracy'] = ba(y, y_pred)

    if pr:
        print(f"\nCoefficients: {results['coefficients']}")
        print(f"\n{color.CYAN}{color.UNDERLINE}{aucroc}{color.END}: {results['auc_roc']}")
        print(f"\n{color.CYAN}{color.UNDERLINE}{ba}{color.END}: {results['balanced_accuracy']}")
        print(f"\nAccuracy: {results['accuracy']:.4f}")

    if save_path:
        with open(save_path, 'w') as f:
            f.write("="*50 + "\n")
            f.write("MODEL EVALUATION RESULTS\n")
            f.write("="*50 + "\n\n")
            
            f.write("COEFFICIENTS:\n")
            f.write(str(results['coefficients']) + "\n\n")
            
            if 'auc_roc' in results:
                f.write(f"AUC-ROC SCORE: {results['auc_roc']:.4f}\n\n")
            
            if 'balanced_accuracy' in results:
                f.write(f"BALANCED ACCURACY: {results['balanced_accuracy']:.4f}\n\n")
            
            f.write(f"ACCURACY: {results['accuracy']:.4f}\n\n")
            
            f.write("="*50 + "\n")
        
        plot_path = str(save_path).replace('.txt', '_confusion_matrix.pdf')
        plt.savefig(plot_path, format='pdf', bbox_inches='tight')
    
    plt.show()

    return results

In [ ]:
def ex(experiment):
    """Method to run single experiment."""

    di = DataInterface(SyntheticDataLoader(experiment.p, experiment.n, experiment.d, experiment.g, random_seed))
    di.split_data(val_size=0.2, test_size=0.3)
    data = di.get_data()
    X_train, y_train = data['train_data'], data['train_labels']
    X_valid, y_valid = data['val_data'], data['val_labels']
    X_test, y_test = data['test_data'], data['test_labels']

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    model = LogisticRegression(penalty=None)
    model.fit(X_train, y_train)
    results_lr = test_model(model, X_test, y_test, ccd=False, pr=False)

    model_ccd = LogRegCCD(verbose=False)
    model_ccd.fit(X_train, y_train, experiment.eps, experiment.lam_max, experiment.lam_count, experiment.k_fold)
    results_ccd = test_model(model_ccd, X_test, y_test, pr=False)

    model_ccd.validate(X_valid, y_valid, measure=aucroc)
    result_ccd_auc_roc = test_model(model_ccd, X_test, y_test, pr=False)

    model_ccd.validate(X_valid, y_valid, measure=ba)
    results_ccd_ba = test_model(model_ccd, X_test, y_test, pr=False)

    return {
        'results_lr': results_lr,
        'results_ccd': results_ccd,
        'result_ccd_auc_roc': result_ccd_auc_roc,
        'results_ccd_ba': results_ccd_ba
    }


In [ ]:
def run_variable_experiments(variable_name, variable_values):
    """Method to run anylysis regarding chosen variable."""

    results = []
    for value in variable_values:
        if variable_name == 'n':
            exp = ExperimentParams(title=f'n_{value}', n=value, p=DEFAULT_P, d=DEFAULT_D, g=DEFAULT_G)
        elif variable_name == 'p':
            exp = ExperimentParams(title=f'p_{value}', n=DEFAULT_N, p=value, d=DEFAULT_D, g=DEFAULT_G)
        elif variable_name == 'd':
            exp = ExperimentParams(title=f'd_{value}', n=DEFAULT_N, p=DEFAULT_P, d=value, g=DEFAULT_G)
        elif variable_name == 'g':
            exp = ExperimentParams(title=f'g_{value}', n=DEFAULT_N, p=DEFAULT_P, d=DEFAULT_D, g=value)
        
        ex_result = ex(exp)
        results.append({
            'variable': variable_name,
            'value': value,
            'results_lr': ex_result['results_lr'],
            'results_ccd': ex_result['results_ccd'],
            'result_ccd_auc_roc': ex_result['result_ccd_auc_roc'],
            'results_ccd_ba': ex_result['results_ccd_ba']
        })
    return results

In [ ]:
def plot_results(variable_results, metric='auc_roc'):
    """Method to plot synthetic data parameters influence analysis"""

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    
    variables = ['n', 'p', 'd', 'g']
    titles = ['Varying n (samples)', 'Varying p (class balance)', 'Varying d (features)', 'Varying g (correlation)']
    
    for i, var in enumerate(variables):
        ax = axes[i]
        var_data = [r for r in variable_results if r['variable'] == var]
        
        x = [d['value'] for d in var_data]
        y_lr = [d['results_lr'][metric] for d in var_data]
        y_ccd = [d['results_ccd'][metric] for d in var_data]
        y_auc_ccd = [d['result_ccd_auc_roc'][metric] for d in var_data]
        y_ba_ccd = [d['results_ccd_ba'][metric] for d in var_data]

        ax.plot(x, y_lr, label='Logistic Regression', marker='o')
        ax.plot(x, y_ccd, label='CCD', marker='s')
        ax.plot(x, y_auc_ccd, label='AUC ROC CCD', marker='s')
        ax.plot(x, y_ba_ccd, label='BA CCD', marker='s')
        ax.set_title(titles[i])
        ax.set_xlabel(var)
        ax.set_ylabel(metric)
        ax.legend()
        ax.grid(True)
    
    plt.tight_layout()
    return fig

## Synthetic data parameters influence analyses

### Experiments ranges

In [ ]:
n_values = np.linspace(100, 10000, num=2, dtype=int) 
p_values = np.linspace(0.1, 0.9, num=2)
d_values = np.linspace(10, 500, num=2, dtype=int)
g_values = np.linspace(0, 0.9, num=2)

### Analyses run

In [ ]:
if run_analyses:
    all_results = []
    all_results.extend(run_variable_experiments('n', n_values))
    all_results.extend(run_variable_experiments('p', p_values))
    all_results.extend(run_variable_experiments('d', d_values))
    all_results.extend(run_variable_experiments('g', g_values))

    fig_auc = plot_results(all_results, metric='auc_roc')
    fig_ba = plot_results(all_results, metric='balanced_accuracy')

    if results_dir:
        os.makedirs(results_dir, exist_ok=True)
        fig_auc.savefig(os.path.join(results_dir, 'auc_roc_comparison.png'), bbox_inches='tight')
        fig_ba.savefig(os.path.join(results_dir, 'balanced_accuracy_comparison.png'), bbox_inches='tight')

# Chosen experiment

## Data loading

In [ ]:
di = DataInterface(SyntheticDataLoader(experiment.p, experiment.n, experiment.d, experiment.g, random_seed))
di.split_data(val_size=0.2, test_size=0.3)
data = di.get_data()
X_train, y_train = data['train_data'], data['train_labels']
X_valid, y_valid = data['val_data'], data['val_labels']
X_test, y_test = data['test_data'], data['test_labels']

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## LogisticRegression

In [ ]:
model = LogisticRegression(penalty=None)
model.fit(X_train, y_train)
_ = test_model(model, X_test, y_test, ccd=False, pr=True)

## LogRegCCD

In [ ]:
model_ccd = LogRegCCD(verbose=False)
model_ccd.fit(X_train, y_train, experiment.eps, experiment.lam_max, experiment.lam_count, experiment.k_fold)
_ = test_model(model_ccd, X_test, y_test, pr=True)

### AUC ROC

In [ ]:
model_ccd.plot(X_valid, y_valid, measure=aucroc)
_ = model_ccd.validate(X_valid, y_valid, measure=aucroc)
_ = test_model(model_ccd, X_test, y_test, pr=True)

### Balanced Accuracy

In [ ]:
model_ccd.plot(X_valid, y_valid, measure=ba)
_ = model_ccd.validate(X_valid, y_valid, measure=ba)
_ = test_model(model_ccd, X_test, y_test, pr=True)